In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time


## LOCAL PATH
PATH = "C:\\Utveksling\\Skole\\Big Data\\Scraper\\resources\\chromedriver.exe"


wd = webdriver.Chrome(PATH)


def get_images_from_google(wd, delay, max_images):
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)

	# open google image -> input what you want -> ctrl+c URL -> ctrl+v URL here:
    url = "INSERTurlHERE.com"

    # load the page
    wd.get(url)

    image_urls = set()
    skips = 0

    while len(image_urls) + skips < max_images:
        scroll_down(wd)

        # get all image thumbnail results
        thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

        for img in thumbnails[len(image_urls) + skips:max_images]:
            
			# try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(delay)
            except:
                continue

            # extract image urls
            images = wd.find_elements(By.CLASS_NAME, "n3VNCb")
            for image in images:
                if image.get_attribute('src') in image_urls:
                    max_images += 1
                    skips += 1
                    break

                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                    print(f"Found {len(image_urls)}")

    return image_urls


def download_image(download_path, url, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

        print("Success")
    except Exception as e:
        print('FAILED -', e)


# (wd, delay, max_images(change it if you want more))
urls = get_images_from_google(wd, 1, 400)

for i, url in enumerate(urls):
    # folder to save img
    download_image("NAME OF FOLDER/", url, str(i) + ".jpg")

wd.quit()